# Decision Tree in Python

In [ ]:
# Import dstools (absolute path required, please change to your systems settings)
import importlib
import sys

path = '/dstools-master/dstools/__init__.py'
name = 'dstools'

spec = importlib.util.spec_from_file_location(name, path)
module = importlib.util.module_from_spec(spec)
sys.modules[spec.name] = module
spec.loader.exec_module(module)

In [ ]:
!pip install graphviz

In [ ]:
import graphviz
import numpy as np
import pandas as pd

from sklearn import tree
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

from dstools.datasets import iris
from dstools.metrics import confusion_matrix, classificationreport

### Datenset vorbereiten

In [ ]:
df = iris()
X = df.drop("Class", axis=1)
y = df["Class"].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33,random_state=147)


### Training

In [ ]:
model = DecisionTreeClassifier(criterion="gini", random_state=147,max_depth=4, min_samples_leaf=2)   
model.fit(X_train,y_train)

### Vorhersage

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
confmat = confusion_matrix(y_pred, y_test, sums=True)
confmat

In [ ]:
print(classificationreport(confmat))

### Visualisierung

In [ ]:
labels = list(df["Class"].unique())
features = list(X.columns)

In [ ]:
features

In [ ]:
data = tree.export_graphviz(model,
                            out_file=None, 
                            feature_names=features,
                            class_names=labels,
                            filled=True,
                            rounded=True,  
                            special_characters=True)  
graph = graphviz.Source(data)  

graph

In [ ]:
#Gini
1-(0.33**2+0.37**2+0.3**2)

In [ ]:
1-((37/67)**2+(30/67)**2)

In [ ]:
1-((37/40)**2+(3/40)**2)

## Hyperparameter

https://scikit-learn.org/stable/modules/tree.html#classification

### Criterion

Die Aufteilung bei einem Knoten erfolgt in sklearn entweder mit 

- Gini Index (*impurity*)
- Information Gain (entropy)

Die Impurity ist ein Maß, wie homogen die Labels bei einem Node sind. Der Informationsgewinn (*information gain*) verwendet die Entropie als Maß und ein Split erfolgt so, dass der Informationsgewinn möglichst groß wird. Der Gini Index hingegen splittet so, dass die kleinsten Unreinheiten (impurities) entstehen.

In der Praxis liefern beide Maßzahlen annähernd gleich gute Ergebnisse, der Gini Index verwendet allerdings keinen Logarithmus zur Berrechnung und ist daher schneller.







### Splitter

Hier kann man zwischen **best** und **random** wählen. 

- best: berechnet die besten Features für den Split, liefert besseres Ergebnis
- random: wählt random Features, braucht weniger Rechenzeit, hilft bei Overfitting

### max_depth

Die theoretische Tiefe eines Baums ist von der Anzahl der Trainingssamples abhängig (und nicht von der Anzahl der Features, da an den gleichen Features mehrmals gesplittet werden kann). Jedoch führen tiefere Bäume zu **Overfitting**.

- None: Der Baum wird solange gesplitted, bis alle Blätter pur sind oder weniger labels als in min_samples_leaf definiert sind haben.

> **Wichtig**: Manche Hyperparameter haben Abhängigkeiten untereinander!

Allgemein gilt, **je tiefer** ein Baum wird, **desto komplexer** wird das Modell (man hat mehr Splits) und es ist mehr Information über die Trainingsdaten im Modell enthalten. Das ist einer der Gründe für Overfitting!

Wenn das Model overfittet, ist die Reduktion der max_depth eine weitere Maßnahme dagegen.